<a href="https://colab.research.google.com/github/sufyan743/cpp/blob/main/Face_recog_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install face_recognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566166 sha256=378ee91873679c5234ff5007c2eebfc857fbd7022907792b00d04ec2ee29d4da
  Stored in directory: /root/.cache/pip/wheels/8f/47/c8/f44c5aebb7507f7c8a2c0bd23151d732d0f0bd6884ad4ac635
Successfully built face-recognition-models


In [2]:
import face_recognition
import json
import os
import numpy as np
from datetime import datetime
from google.colab import files
from google.colab.patches import cv2_imshow
import cv2

# 1. INITIALIZE JSON DATABASE
def initialize_db():
    if not os.path.exists('encodings.json'):
        with open('encodings.json', 'w') as f:
            json.dump({}, f)
    if not os.path.exists('attendance.json'):
        with open('attendance.json', 'w') as f:
            json.dump([], f)

# 2. ENROLL STUDENT (Convert Face to JSON)
def enroll_student(student_name):
    print(f"Please upload an image for {student_name}")
    uploaded = files.upload()
    image_path = list(uploaded.keys())[0]

    image = face_recognition.load_image_file(image_path)
    encoding = face_recognition.face_encodings(image)[0]

    # Load existing JSON data
    with open('encodings.json', 'r') as f:
        data = json.load(f)

    # Convert numpy array to list so it can be saved in JSON
    data[student_name] = encoding.tolist()

    with open('encodings.json', 'w') as f:
        json.dump(data, f)
    print(f"Student {student_name} enrolled successfully in JSON database.")

# 3. RUN AUTOMATED ATTENDANCE
def mark_attendance():
    print("Upload a photo of the classroom/student to mark attendance:")
    uploaded = files.upload()
    image_path = list(uploaded.keys())[0]

    unknown_image = face_recognition.load_image_file(image_path)
    unknown_encodings = face_recognition.face_encodings(unknown_image)

    with open('encodings.json', 'r') as f:
        known_data = json.load(f)

    known_names = list(known_data.keys())
    known_encodings = [np.array(e) for e in known_data.values()]

    found_students = []

    for unknown_enc in unknown_encodings:
        results = face_recognition.compare_faces(known_encodings, unknown_enc, tolerance=0.6)
        if True in results:
            match_index = results.index(True)
            name = known_names[match_index]
            found_students.append(name)

    # Update Attendance JSON
    if found_students:
        with open('attendance.json', 'r') as f:
            logs = json.load(f)

        for name in found_students:
            entry = {"name": name, "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
            logs.append(entry)
            print(f"Attendance marked for: {name}")

        with open('attendance.json', 'w') as f:
            json.dump(logs, f, indent=4)
    else:
        print("No recognized faces found.")

# Start the system
initialize_db()

In [10]:
enroll_student("Your_Name")

Please upload an image for Your_Name


Saving Gemini_Generated_Image_oll0z6oll0z6oll0.png to Gemini_Generated_Image_oll0z6oll0z6oll0 (6).png
Student Your_Name enrolled successfully in JSON database.


In [11]:
mark_attendance()

Upload a photo of the classroom/student to mark attendance:


Saving Gemini_Generated_Image_oll0z6oll0z6oll0.png to Gemini_Generated_Image_oll0z6oll0z6oll0 (7).png
Attendance marked for: Your_Name
